In [1]:
import pandas as pd
import numpy as np
import datetime
from sim import generate_simulation
from model_cancellations import get_otb_res, predict_cancellations

pd.options.display.max_rows = 60
pd.options.display.max_columns = 70

In [2]:
h1_capacity = 187
h2_capacity = 226
AOD = "2017-08-01"
AOD_dt = pd.to_datetime(AOD)

h1_res = pd.read_pickle("pickle/h1_res.pick")
h2_res = pd.read_pickle("pickle/h2_res.pick")
h1_dbd = pd.read_pickle("pickle/h1_dbd.pick")
h2_dbd = pd.read_pickle("pickle/h2_dbd.pick")
h1_sim = pd.read_pickle("pickle/h1_sim.pick")
h2_sim = pd.read_pickle("pickle/h2_sim.pick")

In [10]:
[col for col in h1_sim.columns]

['DOW',
 'RoomsOTB',
 'RevOTB',
 'CxlForecast',
 'Trn_RoomsOTB',
 'Trn_RevOTB',
 'Trn_CxlForecast',
 'TrnP_RoomsOTB',
 'TrnP_RevOTB',
 'TrnP_CxlForecast',
 'Grp_RoomsOTB',
 'Grp_RevOTB',
 'Grp_CxlForecast',
 'Cnt_RoomsOTB',
 'Cnt_RevOTB',
 'Cnt_CxlForecast',
 'Occ',
 'RevPAR',
 'RemSupply',
 'ADR_OTB',
 'Trn_ADR_OTB',
 'TrnP_ADR_OTB',
 'Grp_ADR_OTB',
 'Cnt_ADR_OTB',
 'WE',
 'WD',
 'STLY_Date',
 'LYA_RoomsSold',
 'LYA_ADR',
 'LYA_RoomRev',
 'LYA_RevPAR',
 'LYA_NumCancels',
 'Date',
 'WeekEndDate',
 'SellingPrice',
 'STLY_OTB',
 'STLY_Rev',
 'STLY_ADR',
 'STLY_SellingPrice',
 'STLY_CxlForecast',
 'STLY_TRN_OTB',
 'STLY_TRN_REV',
 'STLY_TRN_CXL',
 'STLY_TRNP_OTB',
 'STLY_TRNP_REV',
 'STLY_TRNP_CXL',
 'STLY_GRP_OTB',
 'STLY_GRP_REV',
 'STLY_GRP_CXL',
 'STLY_CNT_OTB',
 'STLY_CNT_REV',
 'STLY_CNT_CXL']

## Add Pace columns



In [6]:
STLY_COLS = [col for col in h1_sim.columns if col[:5] == 'STLY_']
LYA_COLS = [col for col in h1_sim.columns if col[:4] == 'LYA_']

lam = lambda x: x not in STLY_COLS+LYA_COLS and x[-3:] == 'OTB'
OTB_COLS = [col for col in h1_sim.columns if lam(col)]

In [7]:
OTB_COLS

['RoomsOTB',
 'RevOTB',
 'Trn_RoomsOTB',
 'Trn_RevOTB',
 'TrnP_RoomsOTB',
 'TrnP_RevOTB',
 'Grp_RoomsOTB',
 'Grp_RevOTB',
 'Cnt_RoomsOTB',
 'Cnt_RevOTB',
 'ADR_OTB',
 'Trn_ADR_OTB',
 'TrnP_ADR_OTB',
 'Grp_ADR_OTB',
 'Cnt_ADR_OTB']

In [8]:
STLY_COLS

['STLY_Date',
 'STLY_OTB',
 'STLY_Rev',
 'STLY_ADR',
 'STLY_SellingPrice',
 'STLY_CxlForecast',
 'STLY_TRN_OTB',
 'STLY_TRN_REV',
 'STLY_TRN_CXL',
 'STLY_TRNP_OTB',
 'STLY_TRNP_REV',
 'STLY_TRNP_CXL',
 'STLY_GRP_OTB',
 'STLY_GRP_REV',
 'STLY_GRP_CXL',
 'STLY_CNT_OTB',
 'STLY_CNT_REV',
 'STLY_CNT_CXL']

In [9]:
LYA_COLS

['LYA_RoomsSold', 'LYA_ADR', 'LYA_RoomRev', 'LYA_RevPAR', 'LYA_NumCancels']

In [ ]:
h1_sim["RoomsPace"] = h1_sim.RoomsOTB - h1_sim.STLY_OTB
h1_sim["RoomsPace"] = h1_sim.RoomsOTB - h1_sim.STLY_OTB

## Add T - 30 OTB Features

This will allow us to compare recent pickup vs. STLY